In [40]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")


# Data pre-processing

## load dataset

* unstructure data - notes 
    * extract cheif concern and history info.
 
* structure data - lab and chart



In [52]:
df_sample_notes = pd.read_json('./sepsis_sample_notes.json', lines=True)
df_sample_lab = pd.read_csv('./sepsis_sample_lab.csv')
df_sample_diag = pd.read_csv('./sepsis_sample_diag.csv')

In [606]:
#check distribution
df_sample_diag['long_title'].value_counts()

long_title
Sepsis                                                                                                     71
Pneumonia, unspecified organism                                                                            47
Severe sepsis with septic shock                                                                            36
Cardiogenic shock                                                                                          11
Systemic inflammatory response syndrome (SIRS) of non-infectious origin without acute organ dysfunction     1
Name: count, dtype: int64

## Processing

* extract key info from note. (regexp)
* get patient lab + chartevent
* concat

In [85]:
# processing notes 
#use regexp to extract key info. (primary care)
#print(df_sample_notes['text'][1])

import re

def extract_pre_physical_exam(text):
    """
    Extracts all text before the 'Physical Exam' section.
    """
    match = re.search(r'^(.*?)\n\s*Physical Exam:', text, re.DOTALL | re.IGNORECASE)
    return match.group(1) if match else None

#print(df_sample_notes['text'].apply(extract_pre_physical_exam)[2])
df_sample_notes['extract_notes'] = df_sample_notes['text'].apply(extract_pre_physical_exam)

In [ ]:
#processing lab and chart result. 

df_sample_lab.head(5) #aggregate according to hadm_id
#df_sample_lab_lab = df_sample_lab[df_sample_lab['type']=="lab"]
#df_sample_lab_chart = df_sample_lab[df_sample_lab['type']=="icu_chart"]

#df_sample_lab['type'].value_counts()


def group_hadm_item_js(df):

    # Convert 'charttime' to datetime for sorting
    df['charttime'] = pd.to_datetime(df['charttime'])
    
    # Group by 'hadm_id' and aggregate into nested dictionary format
    def aggregate_lab_results(group):
        grouped_data = {}
        for _, row in group.iterrows():
            time_str = row['charttime'].strftime('%Y-%m-%d %H:%M:%S')
            if time_str not in grouped_data:
                grouped_data[time_str] = {}
            grouped_data[time_str][row['label']] = f"{row['value']} - {row['flag']}" if pd.notna(row['flag']) else row['value']
        return grouped_data
    
    # Apply grouping and sorting
    grouped_df = df.sort_values(by=['charttime']).groupby('hadm_id').apply(aggregate_lab_results).reset_index()
    grouped_df.columns = ['hadm_id', 'item_result']  # Rename columns
    #grouped_df.to_excel('./grouped_df.xlsx')
    return grouped_df

#hadm, json.
df_sample_lab_lab = group_hadm_item_js(df_sample_lab[df_sample_lab['type']=="lab"])
df_sample_lab_chart = group_hadm_item_js(df_sample_lab[df_sample_lab['type']=="icu_chart"])


df_sample_lab_lab["item_result"] = df_sample_lab_lab["item_result"].apply(json.dumps)
df_sample_lab_chart["item_result"] = df_sample_lab_chart["item_result"].apply(json.dumps)


df_sample_lab_lab.head(1)
df_sample_lab_chart.head(1)

In [ ]:
#construct a table , inner join full information. 

df_sample_diag_gp = df_sample_diag.groupby('hadm_id').agg({'long_title': list}).reset_index(drop = False)
df_sample_diag_gp.head(1)


df_sample_result = (df_sample_diag_gp.merge(df_sample_notes[['hadm_id','extract_notes']], how = 'inner', on = 'hadm_id')
.merge(df_sample_lab_lab, how = 'inner', on = 'hadm_id')
.merge(df_sample_lab_chart, how = 'inner', on = 'hadm_id'))

df_sample_result['patient_info'] = df_sample_result['extract_notes'] + df_sample_result['item_result_x'] + df_sample_result['item_result_y']
df_sample_result.head(1)
#result_x is lab 
#result_y is chart

# Query LLM

## Prompt

In [238]:
#concat together. 

concat_diagnosis_prompt = """
You are a sepsis diagnosis expert. Please assess the following patient's data and determine if they have sepsis and its severity. Follow this structured diagnostic reasoning:

Step 1: Sepsis Suspected?
Check if the patient meets Systemic Inflammatory Response Syndrome (SIRS) criteria:

- Temperature <36°C (96.8°F) OR >38°C (100.4°F)
- Heart Rate (HR) >90 bpm
- Respiratory Rate (RR) >20 OR pCO₂ <32 mmHg
- White Blood Cell (WBC) Count >12,000/µL OR <4,000/µL OR >10% Bands
- Sepsis is suspected if at least 2 of the above criteria are met AND there is a confirmed or suspected infection.

Step 2: Severe Sepsis?
If sepsis is suspected, check for organ dysfunction (indicating severe sepsis). The patient has severe sepsis if any of the following are present:

- Neurological: Altered Mental Status
- Circulatory: Systolic Blood Pressure (SBP) <90 mmHg OR Mean Arterial Pressure (MAP) <65 mmHg
- Renal: Creatinine (Cr) >2 mg/dL OR increase of >0.5 mg/dL from baseline
- Liver: Bilirubin >2 mg/dL
- Coagulation: INR >1.5 OR PTT >60 seconds OR Platelets <100,000/uL
- Urine Output: <0.5 mL/kg/hr for >2 hours
- Lactate: Above upper normal limits (e.g., >2 mmol/L)
- Respiratory: Acute respiratory failure requiring invasive/non-invasive ventilation

Step 3: Septic Shock?
If the patient has severe sepsis, check if they meet septic shock criteria:

- SBP <90 mmHg OR MAP <65 mmHg despite adequate fluid resuscitation
- Lactate ≥4 mmol/L, regardless of timing of fluid administration.

Finally, provide a structured assessment like {"Diagnosis":"No Sepsis", "Reason":"Your analysis".}


If sepsis is diagnosed, classify it as:
- No Sepsis (SIRS <2 or no infection)
- Sepsis (SIRS ≥2 + suspected infection)
- Severe Sepsis (Sepsis + organ dysfunction)
- Septic Shock (Severe sepsis + circulatory failure)

Here is the patient infomartion: 

"""

## Model

### Deepseek - r1

In [365]:
from openai import OpenAI


def request_llm(instruct, info):
    
    client = OpenAI(api_key=Deepseek_API_KEY, base_url="https://api.deepseek.com/v1")

    try:
        response = client.chat.completions.create(
            model="deepseek-reasoner",
            messages=[
                {"role": "system", "content": instruct},
                {"role": "user", "content": info},
            ],
            stream=False
            #json_True
        )
        
        return response.choices[0].message.content
    except:
        pass

#result = query_llm_ds(concat_diagnosis_prompt, patient_info )

#print(result.choices[0].message.content)

#request_llm(concat_diagnosis_prompt, df_sample_result.iloc[0,-1]) #test

In [397]:
import hmac
import urllib
import requests
import arrow
import base64
import string
import random
import hashlib
import json
from typing import Dict
import pandas as pd
import concurrent.futures
import re
import time

def update_df_with_api_response(row_index, text, prompt):
    response = request_llm(prompt, text)
    return row_index, response


def dataframe_request(df, text_col, res_col, prompt, num_threads=3):
    """
    Process the DataFrame to make API requests for non-empty, non-None entries in a specified column.

    Params:
    - df (pd.DataFrame): The DataFrame to process.
    - text_col (str): The column from which text needs to be analyzed.
    - prompt (str): System prompt for the API request.
    - num_threads (int, optional): Number of concurrent threads to use, default is 3.
    """
    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = []
        for index, row in df.iterrows():
            # Check if the text column is empty, NaN or None, skip these rows
            if pd.isna(row[text_col]) or row[text_col] == '' or row[text_col] is None:
                continue
            # Check if api_response is empty or NaN, then initiate an API call
            if pd.isna(row[res_col]) or row[res_col] == '':
                futures.append(executor.submit(update_df_with_api_response, index, row[text_col], prompt))

        # Wait for all thread tasks to complete and collect results
        for future in concurrent.futures.as_completed(futures):
            row_index, response = future.result()
            # Update DataFrame
            df.at[row_index, res_col] = response

In [399]:
df_sample_result['result_deepseek'] = ''

In [401]:
#batch request
dataframe_request(df_sample_result, 'patient_info','result_deepseek', concat_diagnosis_prompt)

In [428]:
#df_sample_result.to_excel('./df_sample_result_deepseek.xlsx')
#similar_result
df_sample_result_ds= df_sample_result

### ChatGPT

In [568]:
import os
from openai import OpenAI
from openai import AzureOpenAI  


endpoint = os.getenv("ENDPOINT_URL", "https://chatgpt-simulation.openai.azure.com/")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o-2")  

def request_llm(instruct, info):

    client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",)

    try:
        messages=[
                {"role": "system", "content": instruct},
                {"role": "user", "content": info},
            ]
        completion = client.chat.completions.create(  
        model=deployment,
        messages=messages,
        max_tokens=800,  
        temperature=0.7,  
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
    )
        
        return completion.choices[0].message.content
    except:
        pass

In [455]:
df_sample_result['result_gpt'] = ''

In [570]:
#batch request
dataframe_request(df_sample_result, 'patient_info','result_gpt', concat_diagnosis_prompt)

KeyboardInterrupt: 

In [560]:
#df_sample_result.to_excel('./df_sample_result_gpt.xlsx')
#df_sample_result
df_sample_result_gpt = df_sample_result

# Result Analysis

try to extract key value from dirty json.

In [546]:
import re
import pandas as pd
import json


# Function to clean JSON strings, if needed
def clean_json(json_str):
    # Check if the JSON string is not null
    if pd.notna(json_str):
        # Find the first occurrence of a valid JSON structure
        match = re.search(r'{.*}', json_str, re.DOTALL)
        if match:
            cleaned_json_str = match.group(0)
            # Attempt to load the JSON to see if it's valid
            try:
                # This also helps in removing unnecessary whitespaces
                cleaned_json = json.loads(cleaned_json_str)
                # Convert it back to a string to ensure consistency
                return json.dumps(cleaned_json)
            except json.JSONDecodeError:
                pass  # If JSON is invalid, you might want to handle this case.
    return ''


# Extract key from JSON string
def extract_category_type(json_str, entity):
    try:
        # Load the JSON string into a dictionary
        data = json.loads(json_str)
        # Extract the value for the specified entity
        return data.get(entity, None)
    except json.JSONDecodeError:
        # Return None if there is an error decoding the JSON
        return None

# Extract keys from DataFrame column containing JSON strings
def df_extract_category_type(entity_list, df, extracted_col):
    # Ensure the JSON strings are clean and proper; this step is optional based on your data quality
    df[extracted_col] = df[extracted_col].apply(clean_json)
    
    # For each entity in the list, create a new column in the DataFrame based on the extracted data
    for entity in entity_list:
        # Apply the extraction function across the DataFrame column
        df[entity] = df[extracted_col].apply(lambda x: extract_category_type(x, entity))
    
    return df

In [ ]:
entity_list = ["Reason", "Diagnosis"]
#df_sample_result.head(1)

df1 = df_extract_category_type(entity_list, df_sample_result, 'result_deepseek')
#df1.to_excel('./result_deepseek_extract.xlsx')

df2 = df_extract_category_type(entity_list, df_sample_result, 'result_gpt')
#df2.to_excel('./result_gpt_extract.xlsx')